# Quality Assessment
Now it's your turn. Follow the steps on the platform and use what you've learnt to see how reliable the data is.

In [ ]:
import pandas as pd

Load our cleaned DataFrames

In [ ]:
# orders_cl.csv
url = "https://drive.google.com/file/d/1tXkMOV-F0rBmw8eT2cTLD1EBcMgo5mj4/view?usp=drive_link"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders_cl = pd.read_csv(path)

# orderlines_cl.csv
url = "https://drive.google.com/file/d/1Mv1v7oOexkHRVp5LwPbFBulmEtRT0dv-/view?usp=drive_link"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines_cl = pd.read_csv(path)

In [ ]:
# products_cl.csv
url = "https://drive.google.com/file/d/1VwhUjAfmkr5FPtvhgrG4fGn7bC3XZReT/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products_cl = pd.read_csv(path)

In [ ]:
products_cl =products_cl.copy()

In [ ]:
orderlines_cl =orderlines_cl.copy()

In [ ]:
orders_cl =orders_cl.copy()

In [ ]:
orderlines_cl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216250 entries, 0 to 216249
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   order_id          216250 non-null  int64  
 1   product_quantity  216250 non-null  int64  
 2   sku               216250 non-null  object 
 3   unit_price        216250 non-null  float64
 4   processing_date   216250 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 8.2+ MB


In [ ]:
products_cl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10203 entries, 0 to 10202
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sku       10203 non-null  object 
 1   name      10203 non-null  object 
 2   desc      10197 non-null  object 
 3   price     10157 non-null  float64
 4   in_stock  10203 non-null  int64  
 5   type      10203 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 478.4+ KB


## 1.&nbsp; Define Pandas display format

In [ ]:
# your code here

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_rows', 1000)

## 2.&nbsp; Exclude unwanted orders

2.1 get rid of cancled and shopping cart orders in your data set. They have not been fully purchased yet.

In [ ]:
# your code here
#orders_cl.sample(20)

state_mask = orders_cl['state'].isin(['Cancelled', 'Shopping Basket'])
state_mask.count() # Cancelled and Shopping basket accounted for 226904


NameError: name 'orders_cl' is not defined

In [ ]:
completed_orders_df = orders_cl.loc[~(state_mask)]
completed_orders_df.shape # still have 101862 orders in the document

completed_orders_df.info()
orderlines_cl.info()

2.2 orderlines –> Keep only orders present in orders.

In [ ]:
# difference in rows between completed orders and orderlines

completed_orders_df['order_id'].nunique(), orderlines_cl['order_id'].nunique()
#(101862, 170213) respectively

In [ ]:
orders_orderlines = completed_orders_df.merge(orderlines_cl, how="inner", on="order_id")

In [ ]:
# order_ids that are in both tables
order_ids = orders_orderlines["order_id"].unique()
order_ids

NameError: name 'orders_orderlines' is not defined

In [ ]:
# keep those ids in orders
completed_orders_df = completed_orders_df.loc[completed_orders_df["order_id"].isin(order_ids), :]

# keep those ids in orderlines
orderlines_cl = orderlines_cl.loc[orderlines_cl["order_id"].isin(order_ids), :]

In [ ]:
completed_orders_df["order_id"].nunique(), orderlines_cl["order_id"].nunique()

2.3 orders –> Keep only orders present in orderlines.


In [ ]:
# removing and keeping orders that are present in both orders and orderlines

new_orderlines = orderlines_cl.loc[~orderlines_cl['id_order'].isin(only_in_orderlines)]
new_orderlines

# doubel check to make sure we are not missing anything

set(new_orderlines['id_order'])-set(new_orders['order_id']) # nothing returned it is good
set(new_orders['order_id'])-set(new_orderlines['id_order']) # nothing returned it is good

In [ ]:
# we are working with the following quality dataframes and working on it further
new_orderlines['id_order'].nunique , new_orders['order_id'].nunique

## 3.&nbsp; Exclude orders with unknown products


We are going to try to remove **orders** from the orders table that contains the products id"s that no longer exist there ('sku') This will be done to remove the chance of corrupted data in the dataframe. **I want to remove the orders not just the product number from the data frame because it will effect the entire total price and other quanity information and I can't trust things like that**

In [ ]:
products_cl.info()

NameError: name 'products_cl' is not defined

In [ ]:
orderlines_products = orderlines_cl.merge(products_cl, how="left", on = "sku")[["order_id","sku","name" ]]
orderlines_products.sample(10)


NameError: name 'orderlines_cl' is not defined

In [ ]:
orders_to_delete = orderlines_products.loc[orderlines_products.name.isna(), "order_id"].unique()
orders_to_delete

In [ ]:
orderlines_products.name.isna().value_counts()

In [ ]:
completed_orders_df = completed_orders_df.loc[~completed_orders_df.order_id.isin(orders_to_delete),:]

NameError: name 'completed_orders_df' is not defined

In [ ]:
orderlines_cl = orderlines_cl.loc[~orderlines_cl.order_id.isin(orders_to_delete),:]

In [ ]:
completed_orders_df["order_id"].nunique(), orderlines_cl["order_id"].nunique()

In [ ]:
orderlines_cl["sku"].nunique(), products_cl["sku"].nunique()

In [ ]:
orderlines_cl.info(), completed_orders_df.info()

## 4.&nbsp; Explore the revenue from different tables

#### Step 1:
Create the `unit_price_total` as `orderlines.unit_price` * `orderlines.product_quantity`

In [ ]:
 orderlines_cl_copy = orderlines_cl.copy()
 orders_cl = completed_orders_df.copy()

In [ ]:
orderlines_cl_copy["unit_price_total"] = orderlines_cl_copy["product_quantity"] * orderlines_cl_copy["unit_price"]
orderlines_cl_copy.tail(10)

,order_id,product_quantity,sku,unit_price,processing_date,unit_price_total
216226,527379,1,APP0698,9.99,2018-03-14 13:46:44,9.99
216231,527386,1,APP1922,161.00,2018-03-14 13:49:48,161.00
216235,525853,1,OWC0035-2,71.89,2018-03-14 13:52:18,71.89
216237,527391,1,SAT0008,49.99,2018-03-14 13:53:31,49.99
216238,527392,1,ZAG0024,34.99,2018-03-14 13:53:44,34.99
216240,527394,1,APP1922,161.00,2018-03-14 13:55:15,161.00
216242,527396,1,PAC0653,141.58,2018-03-14 13:55:44,141.58
216243,527384,1,LAC0227,249.99,2018-03-14 13:56:02,249.99
216244,527397,1,JBL0122,42.99,2018-03-14 13:56:38,42.99
216249,527401,1,APP0927,13.99,2018-03-14 13:58:36,13.99


#### Step 2:
Group by `id_order`, summarising by the sum of `unit_price_total`

In [ ]:
# your code here
orderlines_cl_copy["unit_price_total"] = orderlines_qu_copy["product_quantity"] * orderlines_qu_copy["unit_price"]

### What is the average difference between `total_paid` and `unit_price_total`?

In [ ]:
# your code here

### What is the distribution of these differences?

In [ ]:
# your code here

### Can all the differences be explained by shipping costs? If not, what are other plausible explanations?

In [ ]:
# your explanation here

### If there are differences that you can’t explain: what should you do with these orders?

In [ ]:
# your code here

## 5.&nbsp; Become confident about your dataset

Let's hear your thoughts about what you discovered.

In [ ]:
# your thoughts & code here

Do not forget to download your quality controlled DataFrames

In [ ]:
from google.colab import files

#orders_quality_controlled.to_csv("orders_qu.csv", index=False)
#files.download("orders_qu.csv")

#orderlines_quality_controlled.to_csv("orderlines_qu.csv", index=False)
#files.download("orderlines_qu.csv")